## Setup

In [1]:
import duckdb

conn = duckdb.connect('data/duck.db')
conn.sql("""\
INSTALL spatial;
LOAD spatial;
         
INSTALL vss;
LOAD vss;
         
SET hnsw_enable_experimental_persistence = true;
""")


from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic, needs_local_scope)
from IPython.core.magic_arguments import (argument, magic_arguments, parse_argstring)

@magic_arguments()
@argument('query', nargs='*', help="The SQL query to execute")
@argument('--db', '-d', default=None, help="Which connection to use")
@argument('--output', '-o', default=None, help="Name of the variable to store output DataFrame in local scope")
@register_line_cell_magic
@needs_local_scope
def quack(line, cell=None, local_ns=None):
    "Magic that works both as %lcmagic and as %%lcmagic"

    args = parse_argstring(quack, line)
    query = cell if cell else ' '.join(args.query)

    if args.db:
        conn = local_ns[args.db]
    elif 'conn' in local_ns:
        conn = local_ns['conn']
    else:
        
        conn = duckdb.connect(':memory:')
        local_ns['conn'] = conn

    result = conn.sql(query)
    
    if args.output:
        local_ns[args.output] = result
    return result

In [16]:
import ollama
from typing import Optional

def embeddings(prompt:str) -> list[float]:
    result = ollama.embeddings(model='mxbai-embed-large', prompt=prompt)
    return result.embedding

embeds = embeddings("The quick brown fox jumps over the lazy dog.")
print('Embedding Size: ', len(embeds))

Embedding Size:  1024


In [17]:
try:
    conn.remove_function("embeddings")
except:
    # probably didn't exist
    pass

conn.create_function("embeddings", embeddings)

conn.sql(r"""
drop table if exists foo;
create table foo as
    SELECT embeddings('Arizona State University') as embedding;
""")
conn.sql("select * from foo")


┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [20]:
import pandas as pd
import json

def utr_data():
    data = json.loads(open('data/utr-mens.json').read())
    hits = pd.json_normalize(data['hits'])
    utr = pd.DataFrame()

    fields = [
        'id',
        'source.school.name',
        'source.school.shortName',
        'source.school.power6',
        'source.school.power6High',
        'source.school.power6Low',
        'source.school.conference.division.divisionName',
        'source.location.latLng',
        'source.location.cityName',
        'source.location.stateAbbr',
        'source.url',
        'source.memberCount',
    ]
    for field in fields:
        last_field = field.split('.')[-1]
        utr[last_field] = hits[field]

    return utr

utr = utr_data()

conn.sql(r"""
DROP TABLE IF EXISTS utr_vec;
CREATE TABLE utr_vec AS
SELECT
    utr.*,
    embeddings(concat(utr.name, (case when utr.shortName is not null then concat(' (', utr.shortName, ') ') else '' end), ', location: ', utr.cityName, ', ', utr.stateAbbr, ', ', utr.url))::float[1024] AS vectors
FROM utr
""")
conn.sql(r"""
SELECT COUNT(*) AS "Total utr_vec Records" FROM utr_vec
""")

┌───────────────────────┐
│ Total utr_vec Records │
│         int64         │
├───────────────────────┤
│                   956 │
└───────────────────────┘

In [ ]:
schools = pd.read_csv('data/hd2023.csv', encoding='latin1')
charges = pd.read_csv('data/ic2023_ay.csv', encoding='latin1')

costs = conn.sql(r"""
SELECT
    schools.unitid as college_id,
    trim(INSTNM) AS college_name,
    trim(IALIAS) AS short_name,
    regexp_replace(regexp_replace(college_name, '(-.+?ampus$)|(^The )', '', 'g'), '\sat\W(.*)$', ' (\1)') as join_name,
    city AS city,
    stabbr AS state,
    LATITUDE::float as latitude,
    LONGITUD::float as longitude,
    WEBADDR AS url,
    try_cast(CHG3AY3 AS DECIMAL(10,2)) AS total_cost
FROM schools
JOIN charges ON schools.UNITID = charges.UNITID
""")

costs.to_df().to_csv(path_or_buf='data/school_costs.csv', index=False)

conn.sql(r"""
DROP TABLE IF EXISTS costs_vec;
CREATE TABLE costs_vec AS
SELECT
    costs.*,
    embeddings(concat(college_name, (case when short_name is not null then concat(' (', short_name, ') ') else '' end), ', location: ', city, ', ', state, ', ', url))::float[1024] AS vectors,
    null::int as utr_id
FROM costs
""")
conn.sql(r"""
SELECT COUNT(*) as "Total costs_vec Records" FROM costs_vec
""")

┌─────────────────────────┐
│ Total costs_vec Records │
│          int64          │
├─────────────────────────┤
│                    3825 │
└─────────────────────────┘

In [21]:
%%quack
select * from utr_vec where vectors is null;


┌─────────┬─────────┬───────────┬────────┬────────────┬───────────┬──────────────┬──────────┬──────────┬───────────┬─────────┬─────────────┬─────────────┐
│   id    │  name   │ shortName │ power6 │ power6High │ power6Low │ divisionName │  latLng  │ cityName │ stateAbbr │   url   │ memberCount │   vectors   │
│ varchar │ varchar │  varchar  │ double │   double   │  double   │   varchar    │ double[] │ varchar  │  varchar  │ varchar │    int64    │ float[1024] │
├─────────┴─────────┴───────────┴────────┴────────────┴───────────┴──────────────┴──────────┴──────────┴───────────┴─────────┴─────────────┴─────────────┤
│                                                                         0 rows                                                                         │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [23]:
%%quack
create index cost_vec_idx on costs_vec using hnsw (vectors);
create index utr_vec_idx on utr_vec using hnsw (vectors);

In [173]:
%%quack

update costs_vec set utr_id = 1345 where college_id = 110680;
-- 110680 │ University of California-San Diego   │ La Jolla       │ CA      │  32.87775 │ -117.23586 │ www.ucsd.edu/                 │      46042.00 │   NULL │      0.12578559 │ 1345    │ UC San Diego                      │ NULL          │  70.13 │      12.34 │     11.26 │ NCAA - Division I   │ [32.715738, -117.1610838]               │ San Diego      │ CA        │ NULL                                                │         125 │     NULL │          3 │


## Test out `array_cosine_distance` to find similar vectors. 0 = no difference, 1 = very different

In [108]:
%%quack

select
    utr_vec.* exclude (vectors),
    '|' as "|",
    array_cosine_distance(utr_vec.vectors, costs_vec.vectors)::float as cosine_distance,
    costs_vec.* exclude (vectors)
from utr_vec
full join costs_vec on stateAbbr = state
where id in (2803, 2383)
  and array_cosine_distance(utr_vec.vectors, costs_vec.vectors) < 0.3
  and utr_id is null
order by name, array_cosine_distance(utr_vec.vectors, costs_vec.vectors)::float asc
;


┌─────────┬─────────────────────────────────┬─────────────┬────────┬────────────┬───────────┬─────────────────────┬──────────────────────────────────┬──────────────┬───────────┬─────────┬─────────────┬────────┬─────────┬─────────────────┬────────────┬─────────────────────────────────┬────────────┬─────────────────────────────────┬───────────────────┬─────────┬───────────┬────────────┬─────────────────────────────────────┬───────────────┐
│   id    │              name               │  shortName  │ power6 │ power6High │ power6Low │    divisionName     │              latLng              │   cityName   │ stateAbbr │   url   │ memberCount │ utr_id │    |    │ cosine_distance │ college_id │          college_name           │ short_name │            join_name            │       city        │  state  │ latitude  │ longitude  │                 url                 │  total_cost   │
│ varchar │             varchar             │   varchar   │ double │   double   │  double   │       varchar       │ 

In [142]:
%%quack
-- reset all the UTR assignments
UPDATE costs_vec SET utr_id = null;

In [143]:
%%quack

WITH matches AS (
    SELECT
        DISTINCT ON (college_id)
        costs_vec.* exclude (vectors),
        array_cosine_distance(utr_vec.vectors, costs_vec.vectors)::float as cosine_distance,
        utr_vec.* exclude (vectors),
        '|' as "|",
    FROM costs_vec
    LEFT JOIN utr_vec on stateAbbr = state
    WHERE  array_cosine_distance(utr_vec.vectors, costs_vec.vectors) < 0.07
      AND costs_vec.utr_id IS NULL
    ORDER BY college_id, array_cosine_distance(utr_vec.vectors, costs_vec.vectors)::float asc
)
UPDATE costs_vec SET
    utr_id = matches.id
FROM matches
WHERE costs_vec.college_id = matches.college_id
;


In [148]:
%%quack

WITH
unassigned_utr AS (
    SELECT
        utr_vec.*
    FROM utr_vec
    LEFT JOIN costs_vec on costs_vec.utr_id = utr_vec.id
    WHERE costs_vec.utr_id IS NULL
),
matches AS (
    SELECT
        DISTINCT ON (college_id)
        costs_vec.* exclude (vectors),
        array_cosine_distance(utr.vectors, costs_vec.vectors)::float as cosine_distance,
        utr.* exclude (vectors)
    FROM costs_vec
    LEFT JOIN unassigned_utr AS utr on stateAbbr = state
    WHERE  array_cosine_distance(utr.vectors, costs_vec.vectors) < 0.09
      AND costs_vec.utr_id IS NULL
    ORDER BY college_id, array_cosine_distance(utr.vectors, costs_vec.vectors)::float asc
),
counts as (
    select
        name,
        count(1)
    from matches
    group by 1
    having count(1) > 1
)
UPDATE costs_vec SET
    utr_id = matches.id
FROM matches
WHERE costs_vec.college_id = matches.college_id
;


In [169]:
%%quack

WITH
unassigned_utr AS (
    SELECT
        utr_vec.*
    FROM utr_vec
    LEFT JOIN costs_vec on costs_vec.utr_id = utr_vec.id
    WHERE costs_vec.utr_id IS NULL
),
matches AS (
    SELECT
        DISTINCT ON (college_id)
        costs_vec.* exclude (vectors),
        array_cosine_distance(utr.vectors, costs_vec.vectors)::float as cosine_distance,
        utr.* exclude (vectors)
    FROM costs_vec
    LEFT JOIN unassigned_utr AS utr on stateAbbr = state
    WHERE  array_cosine_distance(utr.vectors, costs_vec.vectors) < 0.125
      AND costs_vec.utr_id IS NULL
    ORDER BY college_id, array_cosine_distance(utr.vectors, costs_vec.vectors)::float asc
),
counts as (
    select
        * exclude(short_name, join_name),
        count(college_id) over (partition by name) as name_count
    from matches
)
UPDATE costs_vec SET
    utr_id = matches.id
FROM matches
WHERE costs_vec.college_id = matches.college_id
;


In [174]:
%%quack

WITH
unassigned_utr AS (
    SELECT
        utr_vec.*
    FROM utr_vec
    LEFT JOIN costs_vec on costs_vec.utr_id = utr_vec.id
    WHERE costs_vec.utr_id IS NULL
),
matches AS (
    SELECT
        DISTINCT ON (college_id)
        costs_vec.* exclude (vectors),
        array_cosine_distance(utr.vectors, costs_vec.vectors)::float as cosine_distance,
        utr.* exclude (vectors)
    FROM costs_vec
    LEFT JOIN unassigned_utr AS utr on stateAbbr = state
    WHERE  array_cosine_distance(utr.vectors, costs_vec.vectors) < 0.15
      AND costs_vec.utr_id IS NULL
    ORDER BY college_id, array_cosine_distance(utr.vectors, costs_vec.vectors)::float asc
),
counts as (
    select
        * exclude(short_name, join_name),
        count(college_id) over (partition by name) as name_count
    from matches
)
select * from counts order by name_count desc, name;
;


┌────────────┬──────────────────────────────────────┬────────────────┬─────────┬───────────┬────────────┬────────────────────────────────┬───────────────┬────────┬─────────────────┬─────────┬───────────────────────────────────┬───────────────┬────────┬────────────┬───────────┬─────────────────────┬─────────────────────────────────────────┬────────────────┬───────────┬─────────────────────────────────────────────────────┬─────────────┬──────────┬────────────┐
│ college_id │             college_name             │      city      │  state  │ latitude  │ longitude  │              url               │  total_cost   │ utr_id │ cosine_distance │   id    │               name                │   shortName   │ power6 │ power6High │ power6Low │    divisionName     │                 latLng                  │    cityName    │ stateAbbr │                        url_1                        │ memberCount │ utr_id_1 │ name_count │
│   int64    │               varchar                │    varchar     │ var

In [71]:
%%quack

DROP TABLE IF EXISTS utr_cosine_matches;
CREATE TABLE utr_cosine_matches as
SELECT
    utr_vec.* exclude (vectors),
    array_cosine_distance(utr_vec.vectors, costs_vec.vectors)::float as cosine_distance,
    costs_vec.college_id as other_college_id
FROM utr_vec
LEFT JOIN costs_vec ON utr_vec.stateAbbr = costs_vec.state  -- at least match on state
WHERE array_cosine_distance(utr_vec.vectors, costs_vec.vectors) < 0.5
;


In [72]:
%%quack

DROP TABLE IF EXISTS utr_best_matches;
CREATE TABLE utr_best_matches as
SELECT DISTINCT ON (id)
    *
FROM utr_cosine_matches
ORDER BY id, cosine_distance
;


In [76]:
%%quack
-- │   id    │                  name                  │       shortName       │ power6 │ power6High │ power6Low │    divisionName     │              latLng               │   cityName   │ stateAbbr │                                                url                                                 │ memberCount │ cosine_distance │ other_college_id │ college_id │                    college_name                     │                         short_name                         │                      join_name                      │     city     │  state  │ latitude  │ longitude  │              url              │  total_cost   │

select
    id,
    concat(name, '[ ', shortName, ' ]') as utr_name,
    concat(college_name, '[ ', short_name, ' ]') as ipeds_name,
    cityName,
    city,
    college_id,
    cosine_distance,
from utr_best_matches
LEFT join costs_vec on utr_best_matches.other_college_id = costs_vec.college_id
;


┌─────────┬────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┬──────────────┬────────────┬─────────────────┐
│   id    │                                      utr_name                                      │                                                    ipeds_name                                                     │   cityName   │     city     │ college_id │ cosine_distance │
│ varchar │                                      varchar                                       │                                                      varchar                                                      │   varchar    │   varchar    │   int64    │      float      │
├─────────┼────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────

In [ ]:


utr_costs = conn.sql(r"""
SELECT
    utr.*,
    costs.college_id,
    costs.total_cost
FROM utr
LEFT JOIN costs ON (
    (
        regexp_replace(utr.name, '\W', '', 'g') IN (regexp_replace(costs.join_name, '\W', '', 'g'))
        OR 
        (
            LENGTH(utr.shortName) > 1
            AND LENGTH(costs.short_name) > 1      
            AND regexp_replace(utr.shortName, '\W', '', 'g') IN ( regexp_replace(costs.short_name, '\W', '', 'g'))
        )
    )
    OR
    (
        concat(regexp_replace(utr.name, '\W', '', 'g'), cityName) IN (regexp_replace(costs.join_name, '\W', '', 'g'))
    )
    OR
    (
        LENGTH(utr.shortName) > 1
        AND list_has_any(regexp_split_to_array(short_name, '\s*\|\s*'), [utr.shortName, utr.name, utr.shortName || ' ' || utr.cityName])
    )
    )                     
    AND utr.stateAbbr = costs.state                     
""")

conn.sql("""\
SELECT
    utr.*,
FROM utr_costs AS utr
WHERE utr.college_id IS NULL
""")

In [ ]:
%%quack

SELECT
    *
FROM costs WHERE college_name ilike '%Arizona%University%';

┌────────────┬────────────────────────────────────────────┬─────────────┬────────────────────────────────────────────┬────────────┬─────────┬───────────┬─────────────┬───────────────┐
│ college_id │                college_name                │ short_name  │                 join_name                  │    city    │  state  │ latitude  │  longitude  │  total_cost   │
│   int64    │                  varchar                   │   varchar   │                  varchar                   │  varchar   │ varchar │   float   │    float    │ decimal(10,2) │
├────────────┼────────────────────────────────────────────┼─────────────┼────────────────────────────────────────────┼────────────┼─────────┼───────────┼─────────────┼───────────────┤
│     104151 │ Arizona State University Campus Immersion  │ ASU Tempe   │ Arizona State University Campus Immersion  │ Tempe      │ AZ      │  33.41772 │  -111.93438 │      32193.00 │
│     105330 │ Northern Arizona University                │ NAU         │ Northe

In [ ]:
%%quack

WITH unmatched_costs AS (
  SELECT
    costs.*,
  FROM costs
  LEFT JOIN utr_costs ON costs.college_id = utr_costs.college_id
  WHERE utr_costs.college_id is null
)
SELECT
  utr.*,
  costs.college_name,
  costs.short_name,
  costs.college_id,
  costs.total_cost,
  round(ST_Distance(ST_Point(utr.latLng[1], utr.latLng[2]), ST_Point(costs.latitude, costs.longitude)), 3) as dist
FROM utr_costs AS utr
LEFT JOIN unmatched_costs AS costs ON ST_Distance(ST_Point(utr.latLng[1], utr.latLng[2]), ST_Point(costs.latitude, costs.longitude)) < 0.02
WHERE name ilike '%Arizona%University%'
  AND utr.college_id is null
ORDER BY name, dist
;


┌─────────┬──────────────────────────┬───────────────┬────────┬────────────┬───────────┬───────────────────┬────────────────────────────┬──────────┬───────────┬──────────────────────────────────────────────────┬─────────────┬────────────┬───────────────┬───────────────────────────────────────────┬──────────────────┬────────────┬───────────────┬────────┐
│   id    │           name           │   shortName   │ power6 │ power6High │ power6Low │   divisionName    │           latLng           │ cityName │ stateAbbr │                       url                        │ memberCount │ college_id │  total_cost   │               college_name                │    short_name    │ college_id │  total_cost   │  dist  │
│ varchar │         varchar          │    varchar    │ double │   double   │  double   │      varchar      │          double[]          │ varchar  │  varchar  │                     varchar                      │    int64    │   int64    │ decimal(10,2) │                  varchar         